In [1]:

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from autorocks.data.loader.all_models_result_aggregator import create_all_models_comparison_dataset
from autorocks.dir_struct import LocalResultDir, RootDir
from autorocks.envs.gem5.benchmarks.benchmark_tasks import MachSuiteTask
from autorocks.envs.gem5.schema import Gem5ParametersCollection20

task_name = str(MachSuiteTask.AES)
exp_dir = LocalResultDir / f"gem5/{task_name}/20_params/100_iter"
model_comparison_data = create_all_models_comparison_dataset(exp_dir)

param_space = Gem5ParametersCollection20()
param_names = set([p.name for p in param_space.parameters()])
main_targets = ["bench_stats.avg_power", "detailed_stats.system.sim_seconds"]

In [ ]:
model_comparison_data.sys_observations["detailed_stats.performance"][0]

In [ ]:
from notebooks.gem5.statistics_parser import all_models_parser

perfs_df = all_models_parser(exp_dir)

# Structure timing loop

In [ ]:
# Combine all models data and flatten the structure
system_pref = model_comparison_data.system_performance
model_filter = "Random"
system_pref = system_pref[system_pref.model == model_filter]
system_pref = system_pref[["bench_stats.avg_power", "detailed_stats.system.sim_seconds", "step", "iteration"]]
params_df = model_comparison_data.parameters[model_comparison_data.parameters.model == model_filter].drop(
    columns=["model"]
)
extra_perf = perfs_df.performance[perfs_df.performance.model == model_filter].drop(columns=["model"]).fillna(0)

param_targets = params_df.merge(system_pref, on=["step", "iteration"])

In [ ]:
import torch

assert torch.cuda.is_available()

In [ ]:
from notebooks.gem5.causal_pipeline import structure_learning_timing
import pandas as pd
import torch

torch.set_default_tensor_type(torch.cuda.FloatTensor)
torch.set_default_dtype(torch.float32)


def structure_learning_timing_loop(param_targets, extra_perf) -> pd.DataFrame:
    analysis = []
    stop = 100
    iterations = list(param_targets["iteration"].unique())
    for step in range(5, stop):
        print(f"Analysing at step {step}/{stop}")
        param_target_pre_step = param_targets[param_targets["step"] < step].copy()
        extra_perf_pre_step = extra_perf[extra_perf["step"] < step].copy()

        for i in iterations:
            print(f"Analysing at iteration {i}/{max(iterations)}")
            param_target_pre_step_i = param_target_pre_step[param_target_pre_step["iteration"] == i].copy()
            extra_perf_pre_step_i = extra_perf_pre_step[extra_perf_pre_step["iteration"] == i].copy()
            info = structure_learning_timing(
                param_targets=param_target_pre_step_i,
                perfs=extra_perf_pre_step_i,
                main_targets=main_targets,
                param_names=param_names,
                verbose=True,
            )
            info["iteration"] = i
            analysis.append(info)

    return pd.DataFrame(analysis)


res = structure_learning_timing_loop(param_targets=param_targets, extra_perf=extra_perf)
res

In [ ]:
res.to_csv("structure_learning_timing_stnecil3d.csv")

In [ ]:
import pandas as pd

res = pd.read_csv(
    "/home/salabed/workspace/autorocks/local_execution/gem5paper/stencil_stencil3d/20_params/100_iter/structure_learning_timing_stnecil3d.csv",
    index_col=0,
)

res

In [ ]:
from matplotlib import font_manager

fm = font_manager.json_load("/home/salabed/.cache/matplotlib/fontlist-v330.json")

fm.findfont("serif", rebuild_if_missing=False)
fm.findfont("serif", fontext="afm", rebuild_if_missing=False)

In [ ]:
res.tail(-12)

In [ ]:
initial = pd.DataFrame([{"time": 0, "max_dim": 20, "iteration": 0}] * 15)

new_res = pd.concat([initial, res.tail(-15)])
new_res

In [ ]:
import numpy as np

for iteration in res.iteration.unique():
    res.loc[res["iteration"] == iteration, "step"] = np.arange(
        0, int(len(res[res["iteration"] == iteration])), 1, dtype=int
    )
res

In [ ]:
new_res = res.copy()

new_res.loc[new_res["step"] <= 15, "max_dim"] = 20
new_res.loc[new_res["step"] <= 15, "time"] = 0
new_res

In [ ]:
new_res["cumtime"] = new_res.groupby("iteration")["time"].cumsum()

new_res["cumtime5"] = new_res.groupby("iteration")["time"].cumsum()

In [ ]:
from matplotlib import ticker

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

plt.rcParams["svg.fonttype"] = "none"
plt.rcParams["font.family"] = "Arial"

plt.style.use("ggplot")
sns.set_theme(style="ticks", rc={"axes.spines.right": False, "axes.spines.top": False})
sns.set_context("paper")  # , font_scale=1.5, rc={"lines.linewidth": 1.5})
plt.rc("font", family="sans-serif", serif="Arial")

colors = sns.color_palette("colorblind", 2)
fig, ax_dim = plt.subplots(figsize=(4, 3))
ax_dim = sns.lineplot(
    data=new_res, x="step", y="max_dim", ax=ax_dim, color=colors[0], ls="solid", label="Max Dim", legend=False
)
ax_time = ax_dim.twinx()
ax_time = sns.lineplot(
    data=new_res, x="step", y="cumtime", ax=ax_time, legend=False, color=colors[1], ls="dashed", label="Total Time"
)

for axis in [ax_dim.xaxis, ax_dim.yaxis]:
    axis.set_major_locator(ticker.MaxNLocator(integer=True))
ax_dim.set(xlabel="Step", ylabel="Max Dim")
ax_time.set(xlabel="Step", ylabel="Time(sec)")

fig.legend(loc="upper center")
# lns = ln1+ln2
# labs = [l.get_label() for l in lns]
# ax_dim.legend(lns, labs, loc=0)
ax_dim.axis("tight")
fig.tight_layout()
plt.show()

fig.savefig(
    "/home/salabed/workspace/latex/papers/osdi21_bograph/figs/learning_dim_total.svg",
    bbox_inches="tight",
    format="svg",
    dpi=600,
)